In [77]:
## cria a matriz referente ao sistema de 14 barras

import numpy as np

#Criação da Matriz
sistema_p = np.zeros((14, 14), dtype = complex)

#Impedancias entre barras
sistema_p[0][1] = complex(0,0.0592) 
sistema_p[0][4] = complex(0, 0.223) 
sistema_p[1][2] = complex(0, 0.198) 
sistema_p[1][3] = complex(0, 0.1763) 
sistema_p[1][4] = complex(0, 0.1739) 
sistema_p[2][3] = complex(0, 0.171) 
sistema_p[3][4] = complex(0, 0.0421) 
sistema_p[3][6] = complex(0, 0.2091) 
sistema_p[3][8] = complex(0, 0.5562) 
sistema_p[4][5] = complex(0, 0.252) 
sistema_p[5][10] = complex(0, 0.1989) 
sistema_p[5][11] = complex(0, 0.2558) 
sistema_p[5][12] = complex(0, 0.1303) 
sistema_p[6][7] = complex(0, 0.1762) 
sistema_p[6][8] = complex(0, 0.11) 
sistema_p[8][9] = complex(0, 0.0845) 
sistema_p[8][13] = complex(0, 0.2704) 
sistema_p[9][10] = complex(0, 0.1921)
sistema_p[11][12] = complex(0, 0.1999)
sistema_p[12][13] = complex(0, 0.348)


#impedancias shunt
sistema_p[0][0] = complex(0, 0.1) 
sistema_p[1][1] = complex(0, 0.1) 
sistema_p[7][7] = complex(0, 0.2)

In [78]:
## Define uma funcao que converte coordenadas retangulares para polares

import cmath

def rect_para_polar(num):

    raio, angulo = cmath.polar(num)

    angulo_graus = np.degrees(angulo)

    return raio, angulo_graus
    

In [79]:
## Define uma funcao que cria a matriz de fortescue

def matriz_fortescue():

    fortescue = np.zeros((3,3), dtype = complex)

    for i in range(0, 3):

        fortescue[0][i] = 1
        fortescue[i][0] = 1

    fortescue[1][1] = complex(-0.5, -0.866)
    fortescue[2][2] = complex(-0.5, -0.866)

    fortescue[1][2] = complex(-0.5, 0.866)
    fortescue[2][1] = complex(-0.5, 0.866)

    return fortescue

#Cria a Matriz de Fortescue
fortescue = matriz_fortescue()

In [80]:
## Cria o sistema_0 apenas com algumas alteraçoes no sistema p

def cria_sistema_0(sistema_p):

    sistema_0 = np.zeros((14, 14), dtype = complex)

    sistema_0 = sistema_p

    sistema_0[0][0] = complex(0, 0)
    sistema_0[1][1] = complex(0, 0.04)
    
    return sistema_0

sistema_0 = cria_sistema_0(sistema_p)

In [81]:
#Cria uma funcao e calcula matriz de admitância

def calculo_y_barra(sistema):

    y_barra = np.zeros((14, 14), dtype = complex)

    for i in range(len(sistema)):

        for j in range(len(sistema)):

            sistema[j][i] = sistema[i][j]

    for j in range(len(sistema)):

        for i in range(j+1):

            if j != i:

                if sistema[i][j] > 0:

                    y_barra[i][j] = -1/sistema[i][j]
                    y_barra[j][i] = -1/sistema[i][j]

            if j == i:

                for k in range(14):

                    if sistema[j][k] > 0:

                        y_barra[j][i] = y_barra[j][i] + 1/sistema[j][k]
    
    return y_barra

y_barra_p = calculo_y_barra(sistema_p)
y_barra_0 = calculo_y_barra(sistema_0)

In [82]:
# Obtém a matriz Z a partir da inversa da Y
z_barra_p = np.linalg.inv(y_barra_p)
z_barra_0 = np.linalg.inv(y_barra_0)

In [83]:
#Define a barra de falha
barra_falha = 9


In [84]:
## Define a tensao de pre falta
tensao_pre_falta = complex(0, 1)

### Calculo Fase Terra

In [85]:
## Calcula as correntes de sequencia

i_seq_positiva_ft = tensao_pre_falta/ (2*(z_barra_p[barra_falha - 1][barra_falha - 1]) + z_barra_0[barra_falha-1][barra_falha-1])

i_seq_negativa_ft = i_seq_positiva_ft

i_seq_0_ft = i_seq_positiva_ft

## Calcula as correntes terminais

i_fase_falta_ft = 3*i_seq_positiva_ft #Ou seq + Seq - Seq 0

i_fase_alternativa_ft = 0

### Resposta Fase Terra (corrente)

In [86]:
print("Corrente nas Fase da Falta A: ", rect_para_polar(i_fase_falta_ft))

print("Corrente nas demais fases B e C: ", rect_para_polar(i_fase_alternativa_ft))

Corrente nas Fase da Falta A:  (5.457561817760909, 0.0)
Corrente nas demais fases B e C:  (0.0, 0.0)


In [87]:
## Calcula Tensao de sequencia

v_seq_positiva_ft = tensao_pre_falta - (z_barra_p[barra_falha - 1][barra_falha - 1] * i_seq_positiva_ft)

v_seq_negativa_ft = -(z_barra_p[barra_falha - 1][barra_falha - 1] * i_seq_negativa_ft)

v_seq_0_ft = (-z_barra_0[barra_falha-1][barra_falha-1] * i_seq_0_ft)

In [88]:
## Cria um vetor com essas tensoes para jogar no calculo de matrizes com fortescue em seguida
v_array_seq_ft = np.zeros((3, 1), dtype = complex)

v_array_seq_ft[0][0] = v_seq_0_ft

v_array_seq_ft[1][0] = v_seq_positiva_ft

v_array_seq_ft[2][0] = v_seq_negativa_ft

### Respostas Fase Terra (Tensões)

In [89]:
## Tensoes terminais

v_terminais_ft = np.matmul(fortescue, v_array_seq_ft)

print("Tensão na Fase de Falta A:", rect_para_polar(v_terminais_ft[0][0]))

print("Tensão na Fase B:", rect_para_polar(v_terminais_ft[1][0]))

print("Tensão na Fase C:", rect_para_polar(v_terminais_ft[2][0]))

Tensão na Fase de Falta A: (1.1102230246251565e-16, 90.0)
Tensão na Fase B: (0.9999779997579947, -30.00072778082738)
Tensão na Fase C: (0.9999779997579947, -149.99927221917264)


### Curto Fase Fase

#### O processo é parecido com o que foi feito no Fase Terra muda só algumas etapas teóricas do calculo, por isso não vou comentar todos os blocos

In [90]:
i_seq_positiva_ff = tensao_pre_falta / (2*(z_barra_p[barra_falha - 1][barra_falha - 1]))

i_seq_negativa_ff = -i_seq_positiva_ff

i_seq_0_ff = 0

In [91]:
i_array_seq_ff = np.zeros((3, 1), dtype = complex)

i_array_seq_ff[0][0] = 0

i_array_seq_ff[1][0] = i_seq_positiva_ff

i_array_seq_ff[2][0] = i_seq_negativa_ff

### Resposta Fase Fase (Correntes)

In [92]:
i_terminais_ff = np.matmul(fortescue, i_array_seq_ff)

print("Corrente na fase sem Falta (A): ", rect_para_polar(i_terminais_ff[0][0]), "A")

print("Corrente na fase com Falta (B): ", rect_para_polar(i_terminais_ff[1][0]), "A")

print("Corrente na fase com Falta (C): ", rect_para_polar(i_terminais_ff[2][0]), "A")

Corrente na fase sem Falta (A):  (0.0, 0.0) A
Corrente na fase com Falta (B):  (4.7262485341809475, -90.0) A
Corrente na fase com Falta (C):  (4.7262485341809475, 90.0) A


In [93]:
v_seq_positiva_ff = tensao_pre_falta - (i_seq_positiva_ff*z_barra_p[barra_falha - 1][barra_falha - 1])

v_seq_negativa_ff = v_seq_positiva_ff

v_seq_0 = 0

In [94]:
v_array_seq_ff = np.zeros((3, 1), dtype = complex)

v_array_seq_ff[0][0] = 0

v_array_seq_ff[1][0] = v_seq_positiva_ff

v_array_seq_ff[2][0] = v_seq_negativa_ff

### Resposta Fase Fase Tensões

In [95]:
v_terminais_ff = np.matmul(fortescue, v_array_seq_ff)

print("Tensão na Fase (A): ", v_terminais_ff[0], "V")
print("Tensão na Fase (B): ", v_terminais_ff[1], "V")
print("Tensão na Fase (C): ", v_terminais_ff[1], "V")


Tensão na Fase (A):  [0.+1.j] V
Tensão na Fase (B):  [0.-0.5j] V
Tensão na Fase (C):  [0.-0.5j] V


### Curto Fase Fase Terra

In [96]:
i_seq_positiva_fft = tensao_pre_falta / (z_barra_p[barra_falha - 1][barra_falha - 1] + ((z_barra_p[barra_falha - 1][barra_falha - 1]*z_barra_0[barra_falha - 1][barra_falha - 1]) / (z_barra_p[barra_falha - 1][barra_falha - 1]+z_barra_0[barra_falha - 1][barra_falha - 1])))

i_seq_negativa_fft = -i_seq_positiva_fft * (z_barra_0[barra_falha-1][barra_falha-1]/ (z_barra_p[barra_falha-1][barra_falha-1] + z_barra_0[barra_falha-1][barra_falha-1]))

i_seq_0_fft = i_seq_negativa_fft #Pois seq positiva e negativa sao iguais

In [97]:
i_array_seq_fft = np.zeros((3, 1), dtype = complex)

i_array_seq_fft[0][0] = i_seq_0_fft

i_array_seq_fft[1][0] = i_seq_positiva_fft

i_array_seq_fft[2][0] = i_seq_negativa_fft

#### Resposta Fase Fase Terra (corrente)

In [98]:
i_terminais_fft = np.matmul(fortescue, i_array_seq_fft)

print("Corrente na fase sem Falta (A): ", rect_para_polar(i_terminais_fft[0][0]), "A")

print("Corrente na fase com Falta (B): ", rect_para_polar(i_terminais_fft[1][0]), "A")

print("Corrente na fase com Falta (C): ", rect_para_polar(i_terminais_fft[2][0]), "A")

Corrente na fase sem Falta (A):  (4.440892098500626e-16, 0.0) A
Corrente na fase com Falta (B):  (5.457441750080158, -120.00072778082739) A
Corrente na fase com Falta (C):  (5.457441750080158, 120.00072778082739) A


In [99]:
v_seq_positiva_fft = i_seq_positiva_fft * ((z_barra_p[barra_falha - 1][barra_falha - 1]*z_barra_0[barra_falha - 1][barra_falha - 1]) / (z_barra_p[barra_falha - 1][barra_falha - 1]+z_barra_0[barra_falha - 1][barra_falha - 1]))

v_seq_negativa_fft = - i_seq_negativa_fft * z_barra_p[barra_falha - 1][barra_falha - 1]

v_seq_0_fft = -i_seq_0_fft * z_barra_0[barra_falha - 1][barra_falha - 1]

In [100]:
v_array_seq_fft = np.zeros((3, 1), dtype = complex)

v_array_seq_fft[0][0] = v_seq_0_fft

v_array_seq_fft[1][0] = v_seq_positiva_fft

v_array_seq_fft[2][0] = v_seq_negativa_fft

### Resposta Fase Fase Terra (Tensão)

In [101]:
v_terminais_fft = np.matmul(fortescue, v_array_seq_fft)

print("Tensão na Fase (A): ", v_terminais_fft[0], "V")
print("Tensão na Fase (B): ", v_terminais_fft[1], "V")
print("Tensão na Fase (C): ", v_terminais_fft[1], "V")

Tensão na Fase (A):  [0.+1.j] V
Tensão na Fase (B):  [0.-2.77555756e-17j] V
Tensão na Fase (C):  [0.-2.77555756e-17j] V


In [102]:
## Cálculo da Tensão em todas as demais barras

v_barra_generica_seq_ff = np.zeros((3, 14), dtype = complex)

v_barra_generica = np.zeros((3, 14), dtype = complex)

for n in range(14):
    
    v_barra_generica_seq_ff[1][n] = tensao_pre_falta - (z_barra_p[n][barra_falha - 1]*(tensao_pre_falta/(z_barra_p[n][barra_falha - 1]+z_barra_p[n][barra_falha - 1])))

    v_barra_generica_seq_ff[2][n] = (z_barra_p[n][barra_falha - 1]*(tensao_pre_falta/(z_barra_p[n][barra_falha - 1]+z_barra_p[n][barra_falha - 1])))

    v_barra_generica_seq_ff[0][n] = 0

    v_barra_generica[:, n] = np.matmul(fortescue, v_barra_generica_seq_ff[:,n])

In [103]:
## Amostrando as tensões nas barras de forma organizada

for n in range(14):
    
    print(f"Na Barra: {n} temos as seguintes tensões em cada fase:")
    contador = 0
    for k in ['A', 'B', 'C']:

        print(f"fase {k}: {v_barra_generica[:, n][contador]}")
        contador += 1
        
    

Na Barra: 0 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 1 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 2 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 3 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 4 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 5 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 6 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: (5.551115123125783e-17-0.5j)
fase C: (-5.551115123125783e-17-0.5j)
Na Barra: 7 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j
fase C: -0.5j
Na Barra: 8 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: (5.551115123125783e-17-0.5j)
fase C: (-5.551115123125783e-17-0.5j)
Na Barra: 9 temos as seguintes tensões em cada fase:
fase A: 1j
fase B: -0.5j


In [104]:
## Cálculo da correntes em todos os demais ramos

corrente_sequencia_ramo = np.zeros((14, 14, 3), dtype=complex)
corrente_fase_ramo = np.zeros((14, 14, 3), dtype=complex)


for i in range(14):

    for m in range(i+1):

        if (y_barra_p[i][m] != 0): #Checando se aquele ramo existe
            corrente_sequencia_ramo[i][m][1] = (v_barra_generica_seq_ff[1][i] - v_barra_generica_seq_ff[1][m] / z_barra_p[i][m])
            corrente_sequencia_ramo[i][m][2] = (v_barra_generica_seq_ff[2][i] - v_barra_generica_seq_ff[2][m] / z_barra_p[i][m])

        else: #Caso não exista recebe 0
             corrente_sequencia_ramo[i][m][1] = 0
             corrente_sequencia_ramo[i][m][2] = 0

        if(y_barra_0[i][m] != 0):
        
            corrente_sequencia_ramo[i][m][0] = (v_barra_generica_seq_ff[0][i] - v_barra_generica_seq_ff[0][m] / z_barra_0[i][m])

        else: 
            corrente_sequencia_ramo[i][m][0] = 0


for i in range(14):
    for m in range(i+1):

        corrente_fase_ramo[i][m] = np.matmul(fortescue, corrente_sequencia_ramo[i][m])

#### Cria um loop que visualiza quais as correntes de cada fase, é possível você digitar duas barras e o ramo não existir

In [105]:
# i = 0
# continuar_loop = True

# while i >= -1 and continuar_loop:
#     i_str = input("Insira o número da primeira barra (ou digite -1 para sair): ")

#     if not i_str.isdigit():
#         print("Por favor, insira apenas números.")
#         continue

#     i = int(i_str) - 1

#     if i == -1:
#         print("Saindo do loop.")
#         break

#     m_str = input("Insira o número da segunda barra: ")

#     if not m_str.isdigit():
#         print("Por favor, insira apenas números.")
#         continue

#     m = int(m_str) - 1

#     if (corrente_fase_ramo[0][13][0] == 0) and (corrente_fase_ramo[0][13][1] == 0) and (corrente_fase_ramo[0][13][2] == 0):
#         print("Não existe ramo.")
#     else:
#         print(f"Ramo {i} a {m}")
#         print("Fase A:", corrente_fase_ramo[0][13][0])
#         print("Fase B:", corrente_fase_ramo[0][13][1])
#         print("Fase C:", corrente_fase_ramo[0][13][2])

#     opcao_saida = input("Deseja continuar? (s/n): ")
#     if opcao_saida.lower() != 's':
#         continuar_loop = False


In [106]:
import pandas as pd

In [107]:
### Cria uma tabela de dados que exemplifica a visualizacao dos valores na fase a b e c de corrente para cada possivel ramo

### ATENÇÃO: Apenas os valores da diagonal e abaixo da diagonal são validos, os acima da diagonal são apenas 0, a matriz deveria ser simetrica mas acabou dando preguiça

In [108]:
indices = [f'Barra {i+1}' for i in range(14)]
colunas = [f'Barra {j+1}' for j in range(14)]

df = pd.DataFrame(index=indices, columns=colunas)

for i in range(14):
    for j in range(14):
        df.loc[f'Barra {i+1}', f'Barra {j+1}'] = [corrente_fase_ramo[i, j, 0],
                                                   corrente_fase_ramo[i, j, 1],
                                                   corrente_fase_ramo[i, j, 2]]

df

,Barra 1,Barra 2,Barra 3,Barra 4,Barra 5,Barra 6,Barra 7,Barra 8,Barra 9,Barra 10,Barra 11,Barra 12,Barra 13,Barra 14
Barra 1,"[(-11.708635024166485+1j), (5.8543175120832425...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 2,"[(-27.257467954264957+1j), (13.628733977132478...","[(-26.747738195388532+1j), (13.373869097694266...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 3,"[0j, 0j, 0j]","[(-28.420119444117187+1j), (14.210059722058594...","[(-7.0483679160104735+1j), (3.5241839580052368...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 4,"[0j, 0j, 0j]","[(-30.042351855513978+1j), (15.021175927756989...","[(-15.868333243826113+1j), (7.934166621913057-...","[(-11.27440939843494+1j), (5.63720469921747-0....","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 5,"[(-21.77250296188976+1j), (10.88625148094488-0...","[(-29.365480510667915+1j), (14.682740255333957...","[0j, 0j, 0j]","[(-13.72691768690432+1j), (6.86345884345216-0....","[(-11.029646739313682+1j), (5.514823369656841-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 6,"[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-12.83855517844217+1j), (6.419277589221085-0...","[(-4.321607417734273+1j), (2.1608037088671366-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 7,"[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-15.837330854106176+1j), (7.918665427053089-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-6.8960219552687665+1j), (3.4480109776343837...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 8,"[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-12.971417297860548+1j), (6.485708648930274-...","[(-7.426194698404689+1j), (3.7130973492023447-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 9,"[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-14.671100319417684+1j), (7.335550159708843-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-8.30009865562419+1j), (4.150049327812095-0....","[0j, 0j, 0j]","[(-5.457561817760908+1j), (2.7287809088804544-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"
Barra 10,"[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[(-5.817228745006183+1j), (2.9086143725030915-...","[(-4.244383803431071+1j), (2.1221919017155355-...","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]","[0j, 0j, 0j]"


In [109]:
### Exporta a tabela de dados para uma planilha do excel, apagar a parte acima da diagonal e anexar no trabalho para caso o professor queira ver todos os valores


#df.to_excel('Valores de Corrente.xlsx', index = False)